# Task 2 

### Problem Statement

_Implement a circuit that returns $\lvert01\rangle$ and $\lvert10\rangle$ with equal probability._

Requirements :
- The circuit should consist only of CNOTs, RXs and RYs. 
- Start from all parameters in parametric gates being equal to 0 or randomly chosen. 
- You should find the right set of parameters using gradient descent (you might use more advanced optimization methods if you like). 
- Simulations must be done with sampling - i.e. a limited number of measurements per iteration and noise. 

Compare the results for different numbers of measurements: 1, 10, 100, 1000. 

Bonus question:
How to make sure you produce state $\lvert01\rangle$ + $\lvert10\rangle$ and not $\lvert01\rangle$ - $\lvert10\rangle$ ?


---

In [ ]:
from qiskit import QuantumCircuit, execute, Aer
from qiskit.visualization import plot_histogram as plot_h, plot_state_qsphere as plot_q
from qiskit.quantum_info import Statevector
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

config = {
    'output': 'mpl',
    'backend': Aer.get_backend('statevector_simulator'),
    'shots': 1024
}

In [ ]:
t2_ref_ckt = QuantumCircuit(2)

t2_ref_ckt.x(1)
t2_ref_ckt.h(0)
t2_ref_ckt.cx(0,1)

ref_sv = Statevector.from_label('00').evolve(t2_ref_ckt)
plot_q(ref_sv)

In [ ]:
%autosave 30
import qiskit
qiskit.__qiskit_version__
